<a href="https://colab.research.google.com/github/boritaserey/cosmetic-manufacturer/blob/main/ETL_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pandas faker openpyxl
import sqlite3
import pandas as pd
from google.colab import files
from google.colab import drive
from random import randint, uniform
from datetime import datetime, timedelta
from faker import Faker
import os

In [6]:
try:
    drive.mount('/content/drive')
except:
    pass # Handle case where drive is already mounted


# Create the path if it doesn't exist
db_path = '/content/drive/My Drive/cosmeticETL.db'
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

#here
#Create data with Faker
fake = Faker()

#Connect to sql database
#conn = sqlite3.connect('cosmetics_manufacturer.db')
cursor = conn.cursor()        #cursor fuction is used to retrieve and manipulate data one row at a time

#Step 1: Create Table based on ERD

#Product
cursor.execute('''
CREATE TABLE IF NOT EXISTS Product (
    Product_ID INTEGER PRIMARY KEY,
    Product_Name VARCHAR(30),
    Category VARCHAR(30),
    Size VARCHAR(10),
    Color VARCHAR(30),
    Smell VARCHAR(30),
    Ingredients VARCHAR(255),
    Packaging VARCHAR(30),
    Price DECIMAL(10, 2),
    StockQuantity INTEGER,
    ManufacturingDate DATE,
    Expirationdate DATE,
    Supplier_ID INTERGER
)
''')

#Suppliers
cursor.execute('''
CREATE TABLE IF NOT EXISTS Suppliers (
    Supplier_ID INTEGER PRIMARY KEY,
    Supplier_Name VARCHAR(50),
    Supplier_ContactNumber VARCHAR(15),
    Supplier_Address VARCHAR(100),
    Supplier_City VARCHAR(30),
    Supplier_Country VARCHAR(30),
    ProductsSupplied VARCHAR(50)
)
''')

#Inventory
cursor.execute('''CREATE TABLE IF NOT EXISTS Inventory (
    Inventory_ID INTEGER PRIMARY KEY,
    Product_ID INTEGER,
    WarehouseID INTEGER,
    QuantityInStock INTEGER,
    UnitPrice DECIMAL(15, 2),
    ReorderLevel INTEGER,
    DateReceived DATE
)''')

#Warehouse
cursor.execute('''
CREATE TABLE IF NOT EXISTS Warehouse (
    WarehouseID INTEGER PRIMARY KEY,
    WarehouseLocation VARCHAR(30),
    Capacity_CubicMeters DECIMAL(10, 2),
    Manager_ID INTEGER
)
''')

#Customer
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customer (
    Customer_ID INTEGER PRIMARY KEY,
    Customer_Name VARCHAR(50),
    Customer_Email VARCHAR(50),
    Customer_Address VARCHAR(100),
    Customer_ContactNumber VARCHAR(15),
    LoyaltyStatus ENUM
)
''')

#Order
cursor.execute('''
CREATE TABLE IF NOT EXISTS "Order" (
    Order_ID INTEGER PRIMARY KEY,
    Order_Date DATETIME,
    Customer_ID INTEGER,
    TotalAmount DECIMAL(10, 2),
    ShippingDate DATETIME,
    Order_Status VARCHAR(20)
)
''')

#OrderItem
cursor.execute('''CREATE TABLE IF NOT EXISTS OrderItem (
    OrderItem_ID INTEGER PRIMARY KEY,
    Order_ID INTEGER,
    Product_ID INTEGER,
    Order_Quantity INTEGER,
    UnitPrice DECIMAL(10 ,2)
)''')

#Batch
cursor.execute('''CREATE TABLE IF NOT EXISTS Batch (
    BatchID INTEGER PRIMARY KEY,
    Product_ID INTEGER,
    BatchDate DATE,
    ExpirationDate DATE,
    QuantityProduced INTEGER,
    QualityCheckStatus VARCHAR(20)
)''')

#Manager
cursor.execute('''CREATE TABLE IF NOT EXISTS Manager (
    Manager_ID INTEGER PRIMARY KEY,
    Manager_Name VARCHAR(50),
    Department VARCHAR(30),
    Manager_ContactNumber VARCHAR(15)
)''')

#Step 2: Generate Synthetic Data and Populate Tables

# Define a function to generate a random date
def random_date(start, end):
    return start + timedelta(days=randint(0, (end - start).days))

# Populate Product
products = []
cursor.execute("SELECT COUNT(*) FROM Product")
product_count = cursor.fetchone()[0]
start_id = product_count + 1

for i in range(start_id, start_id + 101):
    products.append((
        i,
        fake.word(),
        fake.word(),
        f"{randint(1, 5)}kg",
        fake.color_name(),
        fake.word(),
        fake.sentence(),
        fake.word(),
        round(uniform(5, 500), 2),
        randint(0, 1000),
        random_date(datetime(2022, 1, 1), datetime(2023, 1, 1)).strftime('%Y-%m-%d'),
        random_date(datetime(2023, 1, 1), datetime(2025, 1, 1)).strftime('%Y-%m-%d'),
        randint(1, 10)
    ))
# Insert data into the Products table
cursor.executemany('''
INSERT INTO Product (Product_ID, Product_Name, Category, Size, Color, Smell, Ingredients, Packaging, Price, StockQuantity, ManufacturingDate, ExpirationDate, Supplier_ID)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', products)

# Populate Suppliers
suppliers = []
cursor.execute("SELECT COUNT(*) FROM Suppliers")
supplier_count = cursor.fetchone()[0]
start_id = supplier_count + 1


for i in range(start_id, start_id + 11):
    suppliers.append((
        i,
        fake.company(),
        fake.phone_number(),
        fake.address(),
        fake.city(),
        fake.country(),
        fake.word()
    ))
# Insert data into the Suppliers table
cursor.executemany('''
INSERT INTO Suppliers (Supplier_ID, Supplier_Name, Supplier_ContactNumber, Supplier_Address, Supplier_City, Supplier_Country, ProductsSupplied)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', suppliers)


# Populate Inventory
cursor.execute("SELECT MAX(Inventory_ID) FROM Inventory") #get the max existing Inventory_ID
max_id = cursor.fetchone()[0]
if max_id is None:
    max_id = 0 #start at 1 if table is empty
start_id = max_id + 1


inventory = []
for i in range(start_id, start_id + 151):  # Assuming 150 inventory records
    inventory.append((
        i,
        randint(1, 100),  # Product_ID (1-100)
        randint(1, 10),  # WarehouseID (1-10)
        randint(0, 1000),  # QuantityInStock (0-1000)
        round(uniform(5, 500), 2),  # UnitPrice (5-500)
        randint(10, 50),  # ReorderLevel (10-50)
        random_date(datetime(2023, 1, 1), datetime(2023, 12, 31)).strftime('%Y-%m-%d')  # DateReceived
    ))
# Insert data into the Inventory table
cursor.executemany('''INSERT INTO Inventory (Inventory_ID, Product_ID, WarehouseID, QuantityInStock, UnitPrice, ReorderLevel, DateReceived) VALUES (?, ?, ?, ?, ?, ?, ?)''', inventory)


#Populate Warehouse
warehouses = []
cursor.execute("SELECT COUNT(*) FROM Warehouse")
warehouse_count = cursor.fetchone()[0]
start_id = warehouse_count + 1

for i in range(start_id, start_id + 11):  # Assuming 10 warehouses
    warehouses.append((
        i,
        fake.city(),  # WarehouseLocation
        round(uniform(100, 1000), 2),  # Capacity_CubicMeters (100 to 1000 cubic meters)
        randint(1, 10)  # Manager_ID (assuming we have 10 managers)
    ))
# Insert data into the Warehouse table
cursor.executemany('''
    INSERT INTO Warehouse (WarehouseID, WarehouseLocation, Capacity_CubicMeters, Manager_ID)
    VALUES (?, ?, ?, ?)
''', warehouses)


# Populate Customers
customers = []
cursor.execute("SELECT COUNT(*) FROM Customer")  # Get the current count of customers
customer_count = cursor.fetchone()[0]
start_id = customer_count + 1

for i in range(start_id, start_id + 51):
    customers.append((
        i,
        fake.name(),
        fake.email(),
        fake.address(),
        fake.phone_number(),
        fake.random_element(elements=('Bronze', 'Silver', 'Gold', 'Platinum'))
    ))
# Insert data into the Customers table
cursor.executemany('''
INSERT INTO Customer (Customer_ID, Customer_Name, Customer_Email, Customer_Address, Customer_ContactNumber, LoyaltyStatus)
VALUES (?, ?, ?, ?, ?, ?)
''', customers)


# Populate Orders
orders = []
# Get the current maximum Order_ID to avoid duplicates
cursor.execute("SELECT MAX(Order_ID) FROM `Order`")
max_order_id = cursor.fetchone()[0]
if max_order_id is None:
    max_order_id = 0  # Start at 1 if the table is empty
start_id = max_order_id + 1


for i in range(start_id, start_id + 101):
    orders.append((
        i,
        random_date(datetime(2023, 1, 1), datetime(2023, 12, 31)).strftime('%Y-%m-%d %H:%M:%S'),
        randint(1, 50),
        round(uniform(20, 2000), 2),
        random_date(datetime(2023, 1, 1), datetime(2024, 1, 1)).strftime('%Y-%m-%d %H:%M:%S'),
        fake.random_element(elements=('Pending', 'Shipped', 'Delivered', 'Cancelled'))
    ))
# Insert data into the Orders table
cursor.executemany('''
INSERT INTO "Order" (Order_ID, Order_Date, Customer_ID, TotalAmount, ShippingDate, Order_Status)
VALUES (?, ?, ?, ?, ?, ?)
''', orders)

#populate OrderItem
order_items = []
# Get the current maximum OrderItem_ID to avoid duplicates
cursor.execute("SELECT MAX(OrderItem_ID) FROM OrderItem")
max_order_item_id = cursor.fetchone()[0]
if max_order_item_id is None:
    max_order_item_id = 0  # Start at 1 if the table is empty
start_id = max_order_item_id + 1

for i in range(start_id, start_id + 201):  # Assuming 200 order items
    order_items.append((
        i,
        randint(1, 100),  # Order_ID (1-100)
        randint(1, 100),  # Product_ID (1-100)
        randint(1, 20),  # Order_Quantity (1-20)
        round(uniform(5, 500), 2)  # UnitPrice (5-500)
    ))
# Insert data into the OrderItem table
cursor.executemany('''INSERT INTO OrderItem (OrderItem_ID, Order_ID, Product_ID, Order_Quantity, UnitPrice) VALUES (?, ?, ?, ?, ?)''', order_items)

#Populate Batch
batches = []
cursor.execute("SELECT MAX(BatchID) FROM Batch")
max_batch_id = cursor.fetchone()[0]
if max_batch_id is None:
    max_batch_id = 0  # Start at 1 if the table is empty
start_id = max_batch_id + 1


for i in range(start_id, start_id + 101):  # Assuming 100 batches
    batches.append((
        i,
        randint(1, 100),  # Product_ID (1-100)
        random_date(datetime(2023, 1, 1), datetime(2023, 6, 30)).strftime('%Y-%m-%d'),  # BatchDate
        random_date(datetime(2024, 1, 1), datetime(2025, 12, 31)).strftime('%Y-%m-%d'),  # ExpirationDate
        randint(500, 5000),  # QuantityProduced (500-5000)
        fake.random_element(elements=('Pass', 'Fail'))  # QualityCheckStatus
    ))
# Insert data into the Batches table
cursor.executemany('''INSERT INTO Batch (BatchID, Product_ID, BatchDate, ExpirationDate, QuantityProduced, QualityCheckStatus) VALUES (?, ?, ?, ?, ?, ?)''', batches)


# Populate Manager
managers = []
# Get the current maximum Manager_ID to avoid duplicates
cursor.execute("SELECT MAX(Manager_ID) FROM Manager")
max_manager_id = cursor.fetchone()[0]
if max_manager_id is None:
    max_manager_id = 0  # Start at 1 if the table is empty
start_id = max_manager_id + 1

for i in range(start_id, start_id + 11):  # Assuming 10 managers
    managers.append((
        i,
        fake.name(),  # Manager_Name
        fake.job(),  # Department
        fake.phone_number()  # Manager_ContactNumber
    ))
# Insert data into the Manager table
cursor.executemany('''INSERT INTO Manager (Manager_ID, Manager_Name, Department, Manager_ContactNumber) VALUES (?, ?, ?, ?)''', managers)

#end db here

conn.commit()







In [7]:
# View the tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

# View data in a specific table (e.g., 'Product')
df = pd.read_sql_query("SELECT * FROM Product;", conn)
print("\nData in 'Product' table:")
print(df)


# Step 1: Extract data
query = "SELECT * FROM Product;"  # Replace 'Product' with the table name you need
df = pd.read_sql_query(query, conn)

# Step 2: Transform data
df['Discounted_Price'] = df['Price'] * 0.9  # Example transformation

# Step 3: Load data (save to CSV)
df.to_csv('transformed_data.csv', index=False)
files.download('transformed_data.csv')  # Download the CSV file

print("ETL pipeline completed!")






Tables in the database:
Product
Suppliers
Inventory
Warehouse
Customer
Order
OrderItem
Batch
Manager

Data in 'Product' table:
     Product_ID Product_Name   Category Size          Color      Smell  \
0             1     discover       true  1kg      Gainsboro  difficult   
1             2       recent   Congress  5kg          Linen      crime   
2             3       rather    feeling  3kg        Thistle     figure   
3             4         free         my  4kg  DarkSlateBlue       fact   
4             5        these       fall  2kg     DarkViolet     simple   
..          ...          ...        ...  ...            ...        ...   
96           97         lose     number  1kg        DimGray     course   
97           98        quite  candidate  2kg     LightCoral       blue   
98           99        space     anyone  2kg        Fuchsia        yet   
99          100     Congress     better  3kg      Gainsboro      front   
100         101        learn       able  4kg          Khaki

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ETL pipeline completed!


In [10]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

# Delete all data in the Product table
cursor.execute("DELETE FROM Product;")
conn.commit()

print("Irrelevant data removed from the Product table.")
conn.close()


Irrelevant data removed from the Product table.


In [12]:
import sqlite3
import random
from datetime import datetime, timedelta

conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()


def random_date(start, end):
    """Generate a random datetime between two datetime objects."""
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return start + timedelta(seconds=random_second)

start_date = datetime(2022, 1, 1)
end_date = datetime.now()

products_data = [
    (100001, "Moisturizing Lotion", "Skincare", "200ml", "No Color", "Herbal",
     "Water, Glycerin, Aloe Vera, Hyaluronic Acid", "Pump Bottle", 25.99,
     random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100002, "Matte Lipstick", "Makeup", "3g", "Red", "Sweet Vanilla",
     "Beeswax, Pigments, Oils, Vitamin E", "Tube", 15.49, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100003, "Eau de Parfum", "Perfume", "50ml", "Transparent", "Floral",
     "Alcohol, Essential Oils, Fragrance Compounds", "Glass Bottle", 89.99, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100004, "Night Cream", "Skincare", "100ml", "No Color", "Mild Fragrance",
     "Glycerin, Retinol, Vitamin C, Shea Butter", "Jar", 35.49, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100005, "Compact Powder", "Makeup", "50g", "Nude", "Unscented",
     "Talc, Pigments, Silica, Mineral Oil", "Compact", 12.99, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100006, "Face Serum", "Skincare", "30ml", "No Color", "Unscented",
     "Hyaluronic Acid, Vitamin C, Niacinamide, Aloe Vera", "Dropper", 45.00, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100007, "Body Spray", "Perfume", "100ml", "No Color", "Fruity",
     "Alcohol, Fragrance Compounds", "Plastic Bottle", 15.99, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100008, "Facial Toner", "Skincare", "250ml", "Baby Pink", "Rose",
     "Niacinamide, Rose Water, Lactic Acid, Gluconolactone", "Glass Bottle", 39.99, random.randint(10, 100), random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100009, "Detox Shampoo", "Haircare", "110ml", "Clear", "Lychee",
     "Hydrolyzed Keratin, Chelating Agents, Aqua, Sodium Chloride", "Plastic Bottle", random.randint(10, 100), 32.00, random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999)),
    (100010, "Body Scrub", "Bodycare", "510g", "Brown", "Coffee",
     "Coconut Oil, Aqua, Coffee Grounds, Sodium C14-16 Olefin Sulfonate", "Jar", random.randint(10, 100), 24.49, random_date(start_date, end_date).strftime('%Y-%m-%d'), random_date(start_date, end_date).strftime('%Y-%m-%d'), random.randint(100001, 199999))
]

cursor.executemany("""
    INSERT INTO Product (Product_ID, Product_Name, Category, Size, Color, Smell, Ingredients, Packaging, Price, StockQuantity, ManufacturingDate, Expirationdate, Supplier_ID)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
""", products_data)

conn.commit()
conn.close()

In [14]:
import sqlite3

# Establish connection
conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

# Now you can perform database operations:
cursor.execute("SELECT Product_ID, Product_Name, Supplier_ID FROM Product;")
rows = cursor.fetchall()

print("Data in 'Product' table with six-digit IDs:")
for row in rows:
    print(row)

# Close the connection when you're done.
conn.close()

Data in 'Product' table with six-digit IDs:
(100001, 'Moisturizing Lotion', 162922)
(100002, 'Matte Lipstick', 145031)
(100003, 'Eau de Parfum', 191444)
(100004, 'Night Cream', 125820)
(100005, 'Compact Powder', 152283)
(100006, 'Face Serum', 117622)
(100007, 'Body Spray', 111914)
(100008, 'Facial Toner', 180672)
(100009, 'Detox Shampoo', 176729)
(100010, 'Body Scrub', 108580)


In [15]:
import pandas as pd

# Sample DataFrame
data = {
    'Order_ID': [101, 102, 103, 101, 102],
    'Product_ID': [1, 2, 3, 1, 2],
    'Order_Item_ID': [1, 2, 3, 4, 5]
}

df = pd.DataFrame(data)

# Update Order_ID to include the year (e.g., 2024)
df['Order_ID'] = df['Order_ID'].apply(lambda x: int(f"2024{x}"))

# Update Product_ID with category-based prefix
def update_product_id(product_id):
    if product_id == 1:  # Example: Lipsticks
        return f"P1{str(product_id).zfill(5)}"
    elif product_id == 2:  # Example: Skincare
        return f"P2{str(product_id).zfill(5)}"
    elif product_id == 3:  # Example: Perfumes
        return f"P3{str(product_id).zfill(5)}"
    else:
        return f"P9{str(product_id).zfill(5)}"  # Default for other categories

df['Product_ID'] = df['Product_ID'].apply(update_product_id)

# Correct Order_Item_ID to six digits
df['Order_Item_ID'] = df.index + 100001  # Start from 100001 and increment

# Display updated DataFrame
print(df)


   Order_ID Product_ID  Order_Item_ID
0   2024101    P100001         100001
1   2024102    P200002         100002
2   2024103    P300003         100003
3   2024101    P100001         100004
4   2024102    P200002         100005


In [16]:
import sqlite3

# Establish connection
conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

# Now you can perform database operations:
cursor.execute("SELECT Product_ID, Product_Name, Supplier_ID FROM Product;")
rows = cursor.fetchall()

print("Data in 'Product' table with six-digit IDs:")
for row in rows:
    print(row)

# Close the connection when you're done.
conn.close()

Data in 'Product' table with six-digit IDs:
(100001, 'Moisturizing Lotion', 162922)
(100002, 'Matte Lipstick', 145031)
(100003, 'Eau de Parfum', 191444)
(100004, 'Night Cream', 125820)
(100005, 'Compact Powder', 152283)
(100006, 'Face Serum', 117622)
(100007, 'Body Spray', 111914)
(100008, 'Facial Toner', 180672)
(100009, 'Detox Shampoo', 176729)
(100010, 'Body Scrub', 108580)


In [20]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

# Delete all data in the Product table
cursor.execute("DELETE FROM Product;")
conn.commit()

print("Irrelevant data removed from the Product table.")
conn.close()

Irrelevant data removed from the Product table.


In [21]:
import pandas as pd

# Sample DataFrame
data = {
    'Order_ID': [101, 102, 103, 101, 102],
    'Product_ID': [1, 2, 3, 1, 2],
    'Order_Item_ID': [1, 2, 3, 4, 5]
}

df = pd.DataFrame(data)

# Update Order_ID to include the year (e.g., 2024)
df['Order_ID'] = df['Order_ID'].apply(lambda x: int(f"2024{x}"))

# Update Product_ID with category-based prefix
def update_product_id(product_id):
    if product_id == 1:  # Example: Lipsticks
        return f"P1{str(product_id).zfill(5)}"
    elif product_id == 2:  # Example: Skincare
        return f"P2{str(product_id).zfill(5)}"
    elif product_id == 3:  # Example: Perfumes
        return f"P3{str(product_id).zfill(5)}"
    else:
        return f"P9{str(product_id).zfill(5)}"  # Default for other categories

df['Product_ID'] = df['Product_ID'].apply(update_product_id)

# Correct Order_Item_ID to six digits
df['Order_Item_ID'] = df.index + 100001  # Start from 100001 and increment

# Display updated DataFrame
print(df)


   Order_ID Product_ID  Order_Item_ID
0   2024101    P100001         100001
1   2024102    P200002         100002
2   2024103    P300003         100003
3   2024101    P100001         100004
4   2024102    P200002         100005


In [25]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

try:
    # Change the data type of Product_ID to VARCHAR
    cursor.execute("ALTER TABLE Product ALTER COLUMN Product_ID TYPE VARCHAR(255);")
    conn.commit()
    print("Product_ID data type changed to VARCHAR successfully.")

except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    # If the column data type is already VARCHAR, you might get an error.
    # Handle the error appropriately (e.g., ignore it or log it).

finally:
    conn.close()

Error: near "ALTER": syntax error


In [30]:
import pandas as pd

# Sample DataFrame
data = {
    'Order_ID': [101, 102, 103, 101, 102],
    'Product_ID': [1, 2, 3, 1, 2],
    'Order_Item_ID': [1, 2, 3, 4, 5]
}

df = pd.DataFrame(data)

# Update Order_ID to include the year (e.g., 2024)
df['Order_ID'] = df['Order_ID'].apply(lambda x: int(f"2024{x}"))

# Update Product_ID with category-based prefix
def update_product_id(product_id):
    if product_id == 1:  # Example: Lipsticks
        return f"P1{str(product_id).zfill(5)}"
    elif product_id == 2:  # Example: Skincare
        return f"P2{str(product_id).zfill(5)}"
    elif product_id == 3:  # Example: Perfumes
        return f"P3{str(product_id).zfill(5)}"
    else:
        return f"P9{str(product_id).zfill(5)}"  # Default for other categories

df['Product_ID'] = df['Product_ID'].apply(update_product_id)

# Correct Order_Item_ID to six digits
df['Order_Item_ID'] = df.index + 100001  # Start from 100001 and increment

# Display updated DataFrame
print(df)


   Order_ID Product_ID  Order_Item_ID
0   2024101    P100001         100001
1   2024102    P200002         100002
2   2024103    P300003         100003
3   2024101    P100001         100004
4   2024102    P200002         100005


In [31]:
cursor.execute("SELECT * FROM Product;")
rows = cursor.fetchall()

print("Data in 'Product' table:")
for row in rows:
    print(row)

Data in 'Product' table:


In [44]:
import sqlite3

conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()

try:
    # 1. Create a temporary table
    cursor.execute("""
        CREATE TABLE Temp_Product (
            Product_ID TEXT PRIMARY KEY,
            Product_Name TEXT,
            Category TEXT,
            Size TEXT,
            Color TEXT,
            Smell TEXT,
            Ingredients TEXT,
            Packaging TEXT,
            Price REAL,
            StockQuantity INTEGER,
            ManufacturingDate DATE,
            Expirationdate DATE,
            Supplier_ID INTEGER
        );
    """)

    # 2. Copy data and apply formatting
    cursor.execute("""
        INSERT INTO Temp_Product
        SELECT 'P' || CAST(Product_ID AS TEXT), Product_Name, Category, Size, Color, Smell, Ingredients, Packaging, Price, StockQuantity, ManufacturingDate, Expirationdate, Supplier_ID
        FROM Product;
    """)

    # 3. Drop the original table
    cursor.execute("DROP TABLE Product;")

    # 4. Rename the temporary table
    cursor.execute("ALTER TABLE Temp_Product RENAME TO Product;")

    conn.commit()
    print("Product_ID data type changed to TEXT and updated successfully.")

except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    # Handle the error appropriately (e.g., ignore it or log it).

finally:
    conn.close()

Product_ID data type changed to TEXT and updated successfully.


In [45]:
conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')
cursor = conn.cursor()


cursor.execute("SELECT * FROM Product;")
rows = cursor.fetchall()

print("Data in 'Product' table:")
for row in rows:
    print(row)

Data in 'Product' table:
('P100001', 'Moisturizing Lotion', 'Skincare', '200ml', 'No Color', 'Herbal', 'Water, Glycerin, Aloe Vera, Hyaluronic Acid', 'Pump Bottle', 25.99, 14, '2022-09-12', '2023-09-17', 148803)
('P100002', 'Matte Lipstick', 'Makeup', '3g', 'Red', 'Sweet Vanilla', 'Beeswax, Pigments, Oils, Vitamin E', 'Tube', 15.49, 31, '2024-02-10', '2023-03-13', 181205)
('P100003', 'Eau de Parfum', 'Perfume', '50ml', 'Transparent', 'Floral', 'Alcohol, Essential Oils, Fragrance Compounds', 'Glass Bottle', 89.99, 17, '2022-09-22', '2023-09-19', 141785)
('P100004', 'Night Cream', 'Skincare', '100ml', 'No Color', 'Mild Fragrance', 'Glycerin, Retinol, Vitamin C, Shea Butter', 'Jar', 35.49, 49, '2024-06-11', '2023-11-03', 181987)
('P100005', 'Compact Powder', 'Makeup', '50g', 'Nude', 'Unscented', 'Talc, Pigments, Silica, Mineral Oil', 'Compact', 12.99, 48, '2023-04-08', '2024-07-05', 181346)
('P100006', 'Face Serum', 'Skincare', '30ml', 'No Color', 'Unscented', 'Hyaluronic Acid, Vitamin C,

In [53]:
import sqlite3
from datetime import datetime

# Drop the existing 'Order' table if it exists
cursor.execute("DROP TABLE IF EXISTS 'Order';")
conn.commit()

# Now, recreate the table with the correct schema
cursor.execute("""
CREATE TABLE IF NOT EXISTS 'Order' (
    Order_ID INTEGER PRIMARY KEY,
    Order_Date DATETIME NOT NULL,
    Customer_ID INTEGER,
    Customer_Name VARCHAR(50) NOT NULL,
    TotalAmount DECIMAL(10, 2) NOT NULL,
    ShippingDate DATETIME NOT NULL,
    Order_Status VARCHAR(20)
);
""")
conn.commit()


orders = [
    (100001, "2024-10-29", 1000, "Alice Johnson", 120.99, "2024-11-01", "Passed"),
    (100002, "2024-10-31", 1001, "Bob Smith", 89.99, "2024-11-02", "Passed"),
    (100003, "2024-11-01", 1002, "Charlie Brown", 45.50, "2024-11-03", "Pending"),
    (100004, "2024-10-31", 1003, "Diana Prince", 75.75, "2024-11-03", "Passed"),
    (100005, "2024-11-02", 1004, "Eve Anderson", 110.25, "2024-11-04", "Passed"),
    (100006, "2024-11-03", 1005, "Frank Miller", 95.50, "2024-11-05", "Pending"),
    (100007, "2024-11-04", 1006, "Grace Wilson", 60.00, "2024-11-05", "Pending"),
]

for order in orders:
    # The column name in the INSERT statement should match the column name in the CREATE TABLE statement.
    cursor.execute("""
        INSERT INTO 'Order' (Order_ID, Order_Date, Customer_ID, Customer_Name, TotalAmount, ShippingDate, Order_Status)
        VALUES (?, ?, ?, ?, ?, ?, ?);
    """, order)

conn.commit()
print("Order data inserted successfully.")

Order data inserted successfully.


In [55]:
# Drop the existing 'OrderItem' table if it exists
cursor.execute("DROP TABLE IF EXISTS OrderItem;")
conn.commit()

# Now, recreate the table with the correct schema
cursor.execute("""
CREATE TABLE IF NOT EXISTS OrderItem (
    Order_Item_ID INTEGER PRIMARY KEY,
    Order_ID INTEGER NOT NULL,
    Product_ID INTEGER NOT NULL,
    Quantity INTEGER NOT NULL,  -- Make sure this column is included
    UnitPrice DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (Order_ID) REFERENCES 'Order' (Order_ID),
    FOREIGN KEY (Product_ID) REFERENCES Product (Product_ID)
);
""")
conn.commit()


orderitems = [
    (100001, 2024101, "P100001", 2, 15.98),  # Moisturizing Lotion
    (100002, 2024102, "P100002", 1, 5.49),  # Matte Lipstick
    (100003, 2024103, "P100003", 1, 49.99),  # Eau de Parfum
    (100004, 2024104, "P100004", 3, 8.97),  # Compact Powder
    (100005, 2024105, "P100005", 1, 25.99),  # Moisturizing Lotion
    (100006, 2024106, "P100006", 2, 49.98)   # Night Cream
]

cursor.execute("PRAGMA table_info(OrderItem);")
print(cursor.fetchall())

cursor.executemany("""
INSERT INTO OrderItem (Order_Item_ID, Order_ID, Product_ID, Quantity, UnitPrice)
VALUES (?, ?, ?, ?, ?);
""", orderitems)
conn.commit()
print("Order_Item data inserted successfully.")

[(0, 'Order_Item_ID', 'INTEGER', 0, None, 1), (1, 'Order_ID', 'INTEGER', 1, None, 0), (2, 'Product_ID', 'INTEGER', 1, None, 0), (3, 'Quantity', 'INTEGER', 1, None, 0), (4, 'UnitPrice', 'DECIMAL(10, 2)', 1, None, 0)]
Order_Item data inserted successfully.


In [57]:
# Drop the existing 'SalesReport' table if it exists
cursor.execute("DROP TABLE IF EXISTS SalesReport;")
conn.commit()

# Recreate the 'SalesReport' table with correct data types
# Change Product_ID to VARCHAR to match the data being inserted
cursor.execute("""
CREATE TABLE IF NOT EXISTS SalesReport (
    Report_ID VARCHAR(255) PRIMARY KEY,  -- Changed to VARCHAR
    Product_ID VARCHAR(255) NOT NULL,    -- Changed to VARCHAR
    QuantitySold INTEGER NOT NULL,
    TotalRevenue DECIMAL(15, 2),
    ReportDate DATE,
    FOREIGN KEY (Product_ID) REFERENCES Product(Product_ID)
);
""")
conn.commit()

salesreports_data = [
    ("P100001", 20, 500.00, "2024-11-01"),
    ("P100002", 15, 375.00, "2024-11-02"),
    ("P100003", 30, 900.00, "2024-11-03"),
    ("P100004", 10, 250.00, "2024-11-04"),
    ("P100005", 25, 625.00, "2024-11-05"),
]

# Add unique Report_ID starting from 001
salesreports = [
    (f"REP{101 + i}", *data) for i, data in enumerate(salesreports_data)
]

# Insert data into the Sales_Report table
cursor.executemany("""
    INSERT INTO SalesReport (Report_ID, Product_ID, QuantitySold, TotalRevenue, ReportDate)
    VALUES (?, ?, ?, ?, ?);
""", salesreports)

# Commit the changes to save the data
conn.commit()
print("SalesReport data inserted successfully.")

SalesReport data inserted successfully.


In [58]:
cursor.execute("SELECT COUNT(*) FROM 'Order';")
row_count = cursor.fetchone()[0]
print(f"Number of rows in Order table: {row_count}")

cursor.execute("SELECT * FROM 'Order' ;")
orders = cursor.fetchall()
print("Orders:")
for order in orders:
    print(order)

Number of rows in Order table: 7
Orders:
(100001, '2024-10-29', 1000, 'Alice Johnson', 120.99, '2024-11-01', 'Passed')
(100002, '2024-10-31', 1001, 'Bob Smith', 89.99, '2024-11-02', 'Passed')
(100003, '2024-11-01', 1002, 'Charlie Brown', 45.5, '2024-11-03', 'Pending')
(100004, '2024-10-31', 1003, 'Diana Prince', 75.75, '2024-11-03', 'Passed')
(100005, '2024-11-02', 1004, 'Eve Anderson', 110.25, '2024-11-04', 'Passed')
(100006, '2024-11-03', 1005, 'Frank Miller', 95.5, '2024-11-05', 'Pending')
(100007, '2024-11-04', 1006, 'Grace Wilson', 60, '2024-11-05', 'Pending')


In [59]:
cursor.execute("SELECT COUNT(*) FROM 'Order';")
row_count = cursor.fetchone()[0]
print(f"Number of rows in Order table: {row_count}")

cursor.execute("SELECT * FROM 'Order' ;")
orders = cursor.fetchall()
print("Orders:")
for order in orders:
    print(order)

Number of rows in Order table: 7
Orders:
(100001, '2024-10-29', 1000, 'Alice Johnson', 120.99, '2024-11-01', 'Passed')
(100002, '2024-10-31', 1001, 'Bob Smith', 89.99, '2024-11-02', 'Passed')
(100003, '2024-11-01', 1002, 'Charlie Brown', 45.5, '2024-11-03', 'Pending')
(100004, '2024-10-31', 1003, 'Diana Prince', 75.75, '2024-11-03', 'Passed')
(100005, '2024-11-02', 1004, 'Eve Anderson', 110.25, '2024-11-04', 'Passed')
(100006, '2024-11-03', 1005, 'Frank Miller', 95.5, '2024-11-05', 'Pending')
(100007, '2024-11-04', 1006, 'Grace Wilson', 60, '2024-11-05', 'Pending')


In [60]:
cursor.execute("SELECT * FROM Product;")
rows = cursor.fetchall()

print("Data in 'Product' table:")
for row in rows:
    print(row)

Data in 'Product' table:
('P100001', 'Moisturizing Lotion', 'Skincare', '200ml', 'No Color', 'Herbal', 'Water, Glycerin, Aloe Vera, Hyaluronic Acid', 'Pump Bottle', 25.99, 14, '2022-09-12', '2023-09-17', 148803)
('P100002', 'Matte Lipstick', 'Makeup', '3g', 'Red', 'Sweet Vanilla', 'Beeswax, Pigments, Oils, Vitamin E', 'Tube', 15.49, 31, '2024-02-10', '2023-03-13', 181205)
('P100003', 'Eau de Parfum', 'Perfume', '50ml', 'Transparent', 'Floral', 'Alcohol, Essential Oils, Fragrance Compounds', 'Glass Bottle', 89.99, 17, '2022-09-22', '2023-09-19', 141785)
('P100004', 'Night Cream', 'Skincare', '100ml', 'No Color', 'Mild Fragrance', 'Glycerin, Retinol, Vitamin C, Shea Butter', 'Jar', 35.49, 49, '2024-06-11', '2023-11-03', 181987)
('P100005', 'Compact Powder', 'Makeup', '50g', 'Nude', 'Unscented', 'Talc, Pigments, Silica, Mineral Oil', 'Compact', 12.99, 48, '2023-04-08', '2024-07-05', 181346)
('P100006', 'Face Serum', 'Skincare', '30ml', 'No Color', 'Unscented', 'Hyaluronic Acid, Vitamin C,

In [61]:
# Reconnect to database
conn = sqlite3.connect('/content/drive/My Drive/cosmeticETL.db')

# Extract Product data
product_data = pd.read_sql_query("SELECT * FROM Product", conn)

# Preview the data
print(product_data.head())

# Close connection
conn.close()

  Product_ID         Product_Name  Category   Size        Color  \
0    P100001  Moisturizing Lotion  Skincare  200ml     No Color   
1    P100002       Matte Lipstick    Makeup     3g          Red   
2    P100003        Eau de Parfum   Perfume   50ml  Transparent   
3    P100004          Night Cream  Skincare  100ml     No Color   
4    P100005       Compact Powder    Makeup    50g         Nude   

            Smell                                   Ingredients     Packaging  \
0          Herbal   Water, Glycerin, Aloe Vera, Hyaluronic Acid   Pump Bottle   
1   Sweet Vanilla            Beeswax, Pigments, Oils, Vitamin E          Tube   
2          Floral  Alcohol, Essential Oils, Fragrance Compounds  Glass Bottle   
3  Mild Fragrance     Glycerin, Retinol, Vitamin C, Shea Butter           Jar   
4       Unscented           Talc, Pigments, Silica, Mineral Oil       Compact   

   Price  StockQuantity ManufacturingDate Expirationdate  Supplier_ID  
0  25.99           14.0        2022-09

In [67]:
import pandas as pd


def transform_data(product_data):
    """
    Transforms the product data by removing duplicates, handling missing values,
    and adding a calculated column (TotalStockValue).

    Args:
        product_data: The input pandas DataFrame.

    Returns:
        The transformed pandas DataFrame.
    """

    # Data cleaning and manipulation
    product_data = product_data.drop_duplicates().copy()  # Remove duplicates and create a copy
    product_data['StockQuantity'] = product_data['StockQuantity'].fillna(0)  # Handle missing values in StockQuantity

    # Add a calculated column (TotalStockValue)
    product_data['TotalStockValue'] = product_data['Price'] * product_data['StockQuantity']

    return product_data  # Return the transformed DataFrame

# Example usage:
transformed_data = transform_data(product_data)  # Call the transform function

# Display the first few rows of the transformed data
print(transformed_data.head())

  Product_ID         Product_Name  Category   Size        Color  \
0    P100001  Moisturizing Lotion  Skincare  200ml     No Color   
1    P100002       Matte Lipstick    Makeup     3g          Red   
2    P100003        Eau de Parfum   Perfume   50ml  Transparent   
3    P100004          Night Cream  Skincare  100ml     No Color   
4    P100005       Compact Powder    Makeup    50g         Nude   

            Smell                                   Ingredients     Packaging  \
0          Herbal   Water, Glycerin, Aloe Vera, Hyaluronic Acid   Pump Bottle   
1   Sweet Vanilla            Beeswax, Pigments, Oils, Vitamin E          Tube   
2          Floral  Alcohol, Essential Oils, Fragrance Compounds  Glass Bottle   
3  Mild Fragrance     Glycerin, Retinol, Vitamin C, Shea Butter           Jar   
4       Unscented           Talc, Pigments, Silica, Mineral Oil       Compact   

   Price  StockQuantity ManufacturingDate Expirationdate  Supplier_ID  \
0  25.99           14.0        2022-0

In [66]:
# Save transformed data to Excel
product_data.to_excel("transformed_product_data.xlsx", index=False)